# Experiments: TensorFlow Lite applications analysis for both with and without HTTP handling implementation

This notebook allows to run every experiments used to analyze the capabilities of the two implmentations of the TensorFlow Lite application, i.e. with and without HTTP handling capability.

In [9]:
# Imports
import os
import re
import sys
import ast
import time
import pathlib
import datetime
import itertools
import subprocess
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [10]:
# Global variables

# Paths
global_res_dir = "./results"
assets_path="./assets"


In [11]:
# ssh and scp functions with arguements
ssh_with_args = "ssh -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no"
scp_with_args = "scp -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no"

## Experiment 1: Analysis of the execution time of each important part of the TensorFlow Lite application for classical setup

In [12]:

# Experiment setup
experiment_name = "Execution time by part"
timestamp = timestamp=datetime.datetime.now() # Current timestamp

iteration = 2
ssh_info = "pptc@192.168.88.3"
ssh_ip = "192.168.88.3"
# implementations = ["label_image", "label_image_with_crow"]
implementations = ["label_image_with_crow"]
mediums = ["WiFi 2.4GHz 20-40MHz", "WiFi 5.0GHz 20-40-80MHz", "Ethernet"]
count_for_usage_script = 30 # correspond to the number of measure taken (1 per second)

### Perform experiment

In [5]:
# Experiment path
exp_folder_name = experiment_name.replace(' ', '_') + "_" + timestamp.strftime("%Y%m%dT%H%M%S")
curr_exp_dir = global_res_dir + "/" + exp_folder_name

# Remote path
remote_exp_results_dir = "~/" + exp_folder_name + "/results"
remote_exp_cpu_usage_dir = remote_exp_results_dir + "/cpu_usage"
remote_exp_ram_usage_dir = remote_exp_results_dir + "/ram_usage"
remote_exp_scripts_dir = "~/" + exp_folder_name + "/scripts"

# Create remote workspace
!{ssh_with_args} {ssh_info} "mkdir -p {remote_exp_cpu_usage_dir}; mkdir -p {remote_exp_ram_usage_dir}"
!{scp_with_args} -r scripts/ {ssh_info}:~/{exp_folder_name}

# Perform the experiment
for imp in implementations:
    imp_mediums = ["na"] if imp == "label_image" else mediums

    # Copy needed assets (executable and images)
    print(f"Copy assets for {imp} on remote device")
    !{scp_with_args} -r {assets_path}/{imp} {ssh_info}:~/{exp_folder_name}/{imp}

    for med in imp_mediums:
        # Validate config:
        print(f"Prepare next config:\n\tImplementation: {imp}\n\tMedium: {med}\n\t\n")
        time.sleep(2)
        v = input("Press ENTER to run the benchmark")
        print(f"Value : {v}")
        if v == "s": # stop
            break
        print("Run benchmark...")


        # Implementation paths
        curr_exp_imp_dir = curr_exp_dir + "/" + imp.replace(' ', '_')
        curr_exp_imp_med_dir = curr_exp_imp_dir + ("/" + med.replace(' ', '_') if med != "na" else "")
        curr_exp_imp_raw_dir = curr_exp_imp_med_dir + "/raw_results"
        curr_exp_imp_raw_file = curr_exp_imp_raw_dir + "/execution_time_by_part.txt"
        curr_exp_imp_res_dir = curr_exp_imp_med_dir + "/results"

        # Remote paths
        cpu_usage_remote_file = remote_exp_cpu_usage_dir + "/results_cpu_usage_" + imp + "_" + med.replace(' ', '_') + ".txt"
        ram_usage_remote_file = remote_exp_ram_usage_dir + "/results_ram_usage_" + imp + "_" + med.replace(' ', '_') + ".txt"

        # Create results dir
        path = Path(curr_exp_dir)
        path.mkdir(parents=True, exist_ok=True)
        path = Path(curr_exp_imp_dir)
        path.mkdir(parents=True, exist_ok=True)
        path = Path(curr_exp_imp_raw_dir)
        path.mkdir(parents=True, exist_ok=True)
        path = Path(curr_exp_imp_res_dir)
        path.mkdir(parents=True, exist_ok=True)

        # Print experiment setup in results file
        # Local for execution time
        with open(curr_exp_imp_raw_file, "w") as raw:
            raw.write(f"# SSH_INFO={ssh_info}\n")
            raw.write(f"# ITERATIONS={iteration}\n")
            raw.write(f"# IMPLEMENTATION={imp}\n")
            raw.write(f"# MEDIUM={med}\n")

        # Remote for cpu and ram usage
        !{ssh_with_args} {ssh_info} "echo -e \"# SSH_INFO={ssh_info}\n# ITERATIONS={iteration}\n# IMPLEMENTATION={imp}\n# MEDIUM={med}\n\" > {cpu_usage_remote_file}"
        !{ssh_with_args} {ssh_info} "echo -e \"# SSH_INFO={ssh_info}\n# ITERATIONS={iteration}\n# IMPLEMENTATION={imp}\n# MEDIUM={med}\n\" > {ram_usage_remote_file}"

        # Perform test
        for i in range(iteration):
            # Print iteration
            print(f"Iteration {i}")
            with open(curr_exp_imp_raw_file, "a") as raw:
                raw.write(f"\n# ITERATION {i}\n")
            !{ssh_with_args} {ssh_info} "echo -e \"\n# ITERATION {i}\n\" >> {cpu_usage_remote_file}"
            !{ssh_with_args} {ssh_info} "echo -e \"\n# ITERATION {i}\n\" >> {ram_usage_remote_file}"

            # Run cpu and ram usage script
            print("Start CPU and RAM usage scripts")
            
            cpu_usage_process = subprocess.Popen(f'{ssh_with_args} {ssh_info} "{remote_exp_scripts_dir}/cpu_usage_script.sh {count_for_usage_script} {cpu_usage_remote_file} 0"', shell=True)
            ram_usage_process = subprocess.Popen(f'{ssh_with_args} {ssh_info} "{remote_exp_scripts_dir}/ram_usage_script.sh {count_for_usage_script} {ram_usage_remote_file} 0"', shell=True)

            
            print("Perform measure")
            if imp == "label_image":
                start_timestamp = int(datetime.datetime.now().timestamp()) # Timestamp at which the inference request started
                !{ssh_with_args} {ssh_info} "cd ~/{exp_folder_name}/{imp}; ./{imp} -m mobilenet_v1_1.0_224.tflite -l labels.txt -i grace_hopper.bmp -t 1 -d 1 2>&1 &" >> {curr_exp_imp_raw_file}

                stop_timestamp = int(datetime.datetime.now().timestamp()) # Timestamp at which the inference request started
            else:
                server_process = subprocess.Popen(f'{ssh_with_args} {ssh_info} "cd ~/{exp_folder_name}/{imp}; ./{imp} -c config.json"', shell=True)
                time.sleep(2)
                
                start_timestamp = int(datetime.datetime.now().timestamp()) # Timestamp at which the inference request started
                !curl -X POST -F "image=@{assets_path}/label_image_with_crow/grace_hopper.bmp" http://{ssh_ip}:18080/label_image >> {curr_exp_imp_raw_file}

                stop_timestamp = int(datetime.datetime.now().timestamp()) # Timestamp at which the inference request started
                
                # Kill server
                print("Kill server")
                server_process.kill()
                !{ssh_with_args} {ssh_info} 'kill $(pidof {imp})'


            print("Sleep")
            time.sleep(2)

            print("Kill CPU and RAM usage scripts")
            cpu_usage_process.kill()
            ram_usage_process.kill()
            !{ssh_with_args} {ssh_info} 'kill $(pidof cpu_usage_script.sh)'
            !{ssh_with_args} {ssh_info} 'kill $(pidof ram_usage_script.sh)'

            !{ssh_with_args} {ssh_info} "echo \"# TIMESTAMP_START={start_timestamp}\" >> {cpu_usage_remote_file}; echo \"# TIMESTAMP_STOP={stop_timestamp}\" >> {cpu_usage_remote_file}"
            !{ssh_with_args} {ssh_info} "echo \"# TIMESTAMP_START={start_timestamp}\" >> {ram_usage_remote_file}; echo \"# TIMESTAMP_STOP={stop_timestamp}\" >> {ram_usage_remote_file}"

        print("Retrieve usage results and remove remote workspace")
        !{scp_with_args} -r {ssh_info}:{remote_exp_cpu_usage_dir} {curr_exp_imp_raw_dir}
        !{scp_with_args} -r {ssh_info}:{remote_exp_ram_usage_dir} {curr_exp_imp_raw_dir}

# Remove remote workspace
!{ssh_with_args} {ssh_info} "rm -rf ~/{exp_folder_name}"

    

cpu_usage_script.sh                           100% 1189   589.8KB/s   00:00    
ram_usage_script.sh                           100% 1253   735.4KB/s   00:00    
Copy assets for label_image_with_crow on remote device
label_image_with_crow                         100% 5424KB   5.4MB/s   00:00    
mobilenet_v1_1.0_224.tflite                   100%   16MB   4.1MB/s   00:03    
grace_hopper.bmp                              100%  919KB  11.6MB/s   00:00    
labels.txt                                    100%   10KB   2.2MB/s   00:00    
config.json                                   100%  499   315.2KB/s   00:00    
Prepare next config:
	Implementation: label_image_with_crow
	Medium: WiFi 2.4GHz 20-40MHz
	

Value : 
Run benchmark...
Iteration 0
Start CPU and RAM usage scripts
Perform measure


INFO: Loaded model ./mobilenet_v1_1.0_224.tflite
INFO: resolved reporter
(2024-08-10 19:48:58) [INFO    ] Crow/master server is running at http://0.0.0.0:18080 using 4 threads
(2024-08-10 19:48:58) [INFO    ] Call `app.loglevel(crow::LogLevel::Warning)` to hide Info level logs.


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:21 --:--:--     0^C
Kill server
Sleep


(2024-08-10 19:49:22) [INFO    ] Closing IO service 0xb57f76f0
(2024-08-10 19:49:22) [INFO    ] Closing IO service 0xb57f76f4
(2024-08-10 19:49:22) [INFO    ] Closing IO service 0xb57f76f8
(2024-08-10 19:49:22) [INFO    ] Closing main IO service (0xb5d254dc)
(2024-08-10 19:49:22) [INFO    ] Exiting.


Kill CPU and RAM usage scripts
Iteration 1
Start CPU and RAM usage scripts
Perform measure


INFO: Loaded model ./mobilenet_v1_1.0_224.tflite
INFO: resolved reporter
(2024-08-10 19:49:27) [INFO    ] Crow/master server is running at http://0.0.0.0:18080 using 4 threads
(2024-08-10 19:49:27) [INFO    ] Call `app.loglevel(crow::LogLevel::Warning)` to hide Info level logs.


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     0

In [17]:

ssh_ip="192.168.88.3"
# process = subprocess.Popen(f'ssh pptc "cd ~/label_image_with_crow; ./label_image_with_crow -c config.json"', shell=True)
# time.sleep(2)
!time curl -X POST -F "image=@{assets_path}/label_image_with_crow/grace_hopper.bmp" http://{ssh_ip}:18080/label_image >> pomme.txt



INFO: Loaded model ./mobilenet_v1_1.0_224.tflite
INFO: resolved reporter
(2024-08-10 19:54:54) [INFO    ] Crow/master server is running at http://0.0.0.0:18080 using 4 threads
(2024-08-10 19:54:54) [INFO    ] Call `app.loglevel(crow::LogLevel::Warning)` to hide Info level logs.


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

(2024-08-10 19:54:56) [INFO    ] Request: 172.16.42.2:53544 0xb5cdd210 HTTP/1.1 POST /label_image


100  919k  100  1075  100  918k    997   852k  0:00:01  0:00:01 --:--:--  854k

real	0m1.087s
user	0m0.000s
sys	0m0.011s


(2024-08-10 19:54:57) [INFO    ] Response: 0xb5cdd210 /label_image 200 0


### Process results

In [ ]:
def process_part_execution_time_raw_results_to_dataframe(raw_results_path):
    df = pd.DataFrame()
    medium=""
    implementation=""

    parts = []
    subparts = []
    values = []

    with open(raw_results_path, "r") as raw:
        for line in raw:
            if line.startswith("# MEDIUM="):
                medium = line.split('=')[1].strip()
            elif line.startswith("# IMPLEMENTATION="):
                implementation = line.split('=')[1].strip()
            elif line.startswith("TIME: "):
                part, subpart, value = line.split(':').strip()
                value = float(value.split(' ')[0].strip())
                parts.append(part)
                subparts.append(subpart)
                values.append(value)

    df["Part"] = parts
    df["Subpart"] = subparts
    df["Execution Time (in us)"] = values
    df["Medium"] = [medium] * df.shape[0]
    df["Implementation"] = [implementation] * df.shape[0]

    return df


In [ ]:
exp_dir = global_res_dir + "/" + experiment_name.replace(' ', '_') + "_" + timestamp.strftime("%Y%m%dT%H%M%S")

df_execution_time_by_part = []

for imp in implementations:
    exp_imp_dir = exp_dir + "/" + imp
    imp_mediums = ["na"] if imp == "label_image" else mediums
    for med in imp_mediums:
        exp_imp_med_dir = exp_imp_dir + ("/" + med.replace(' ', '_') if med != "" else "")
        exp_imp_med_raw_dir = exp_imp_med_dir + "/raw_results"
        exp_imp_med_res_dir = exp_imp_med_dir + "/results"

        path = Path(exp_imp_med_res_dir)
        path.mkdir(parents=True, exist_ok=True)

        df_execution_time_by_part_imp_med = []

        for file in os.listdir(exp_imp_med_raw_dir):
            df_execution_time_by_part_imp_med.append(process_part_execution_time_raw_results_to_dataframe(exp_imp_med_raw_dir + "/" + file))

        df_execution_time_by_part_imp_med = pd.concat(df_execution_time_by_part_imp_med, ignore_index=True)
        df_execution_time_by_part.append(df_execution_time_by_part_imp_med)

        df_execution_time_by_part_imp_med.to_csv(exp_imp_med_res_dir + "/execution_time_by_parts_" + imp + "_" + med + ".csv", index=False)

df_execution_time_by_part = pd.concat(df_execution_time_by_part, ignore_index=True)
df_execution_time_by_part.to_csv(exp_dir + "/execution_time_by_parts.csv", index=False)

display(df_execution_time_by_part)

### Make Graphs

In [ ]:
# Global variables
experiment_name = "Execution time by part" # ! Check that it's the same as in the experiment setup
exp_dir = global_res_dir + "/" + experiment_name.replace(' ', '_') + "_" + timestamp.strftime("%Y%m%dT%H%M%S")

# exp_dir = global_res_dir + "/" + experiment_name.replace(' ', '_') + "_Correct"

graphs_dir = exp_dir + "/graphs"

path = Path(graphs_dir)
path.mkdir(parents=True, exist_ok=True)

for imp in implementations:
    df_imp = df[df["Implementation"] == imp]

    medium_group = df_imp.groupby

In [ ]:
df = pd.read_csv(exp_dir + "/execution_time_by_parts.csv")

for imp in implementations:
    df_imp = df[df["Implementation"] == imp]

    medium_group = df_imp.groupby("Medium")

    colors = sns.color_palette('rocket')[::-1]

    for key, item in medium_group:
        plt.figure(figsize=(10, 6))
        sns.set_style("whitegrid")
        sns.barplot(data=item, x='Subpart', y='Execution Time (in us)', hue='Part', palette=colors)

        plt.ylim(bottom=0)
        plt.tight_layout()
        plt.show